In [1]:
import numpy as np
import GEOparse as gp
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from IPython.display import display

We download the data from the GSE dataset

In [2]:
gse = gp.get_GEO(geo="GSE157239", destdir="./")

11-Nov-2020 21:03:24 DEBUG utils - Directory ./ already exists. Skipping.
11-Nov-2020 21:03:24 INFO GEOparse - File already exist: using local version.
11-Nov-2020 21:03:24 INFO GEOparse - Parsing ./GSE157239_family.soft.gz: 
11-Nov-2020 21:03:24 DEBUG GEOparse - DATABASE: GeoMiame
11-Nov-2020 21:03:24 DEBUG GEOparse - SERIES: GSE157239
11-Nov-2020 21:03:24 DEBUG GEOparse - PLATFORM: GPL21572
/usr/local/lib/python3.9/site-packages/GEOparse/GEOparse.py:110: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  return parse_GSE(filepath, open_kwargs=open_kwargs)
11-Nov-2020 21:03:27 DEBUG GEOparse - SAMPLE: GSM4759790
11-Nov-2020 21:03:27 DEBUG GEOparse - SAMPLE: GSM4759791
11-Nov-2020 21:03:27 DEBUG GEOparse - SAMPLE: GSM4759792
11-Nov-2020 21:03:27 DEBUG GEOparse - SAMPLE: GSM4759793
11-Nov-2020 21:03:27 DEBUG GEOparse - SAMPLE: GSM4759794
11-Nov-2020 21:03:27 DEBUG GEOparse - SAMPLE: GSM4759795
11-Nov-2020 21:03:27 DEBUG GEOparse - SAMPLE

In [3]:
alpha = 0.01


control_columns = np.array([0, 1, 2, 4, 6, 9, 11, 15])
c = np.zeros(16, dtype=bool)
c[control_columns] = 1
values = []
for k, v in gse.gsms.items():
    #print(k)
    ids = v.table["ID_REF"].to_numpy()
    values.append(v.table['VALUE'].to_numpy())
values = np.stack(values, axis=-1)
mask = np.broadcast_to(c, (len(values), 16))

controls = values[mask].reshape(-1, 8)
samples = values[~mask].reshape(-1, 8)
f_stats = stats.f_oneway(controls, samples, axis=-1)
num_significant = np.sum(f_stats.pvalue < alpha)
signif_ids = ids[f_stats.pvalue < alpha]

ids = np.array(ids)
order = np.argsort(f_stats.pvalue)
sorted_ids = ids[order]

In [35]:
raw_data = gse.gpls['GPL21572'].table
signifs = []
for sig_id in signif_ids:
    signifs.append(raw_data[raw_data["ID"] == sig_id])
signifs = pd.concat(signifs)
signifs.insert(16, "P-value", f_stats.pvalue[f_stats.pvalue < alpha])
mirna_mask = signifs['Sequence Type'] == 'miRNA'
mirna_signifs = signifs[mirna_mask]
print(len(mirna_signifs))
display(mirna_signifs)


20


,ID,Accession_ID,Accession,Transcript ID(Array Design),Sequence Type,Species Scientific Name,Alignments,Sequence Length,Sequence,Genome Context,Clustered miRNAs within 10kb,Target Genes,GeneChip Array,Annotation Date,Sequence Source,miRNA_ID,P-value,SPOT_ID
474,20500474,MIMAT0026476_st,MIMAT0026476,hsa-miR-215-3p,miRNA,Homo sapiens,chr1:220291219-220291241 (-),23,UCUGUCAUUUCUUUAGGCCAAUA,ENST00000302637 // antisense // intron // 12 /...,hsa-mir-215 // chr1:220291195-220291304 (-) //...,---,miRNA-4_0,"May 13, 2014",miRBase,hsa-miR-215-3p,0.008630,NaN
1225,20501225,MIMAT0001621_st,MIMAT0001621,hsa-miR-369-5p,miRNA,Homo sapiens,chr14:101531943-101531964 (+),22,AGAUCGACCGUGUUAUAUUCGC,ENST00000362155 // sense // exon // 1,hsa-mir-154 // chr14:101526092-101526175 (+) /...,MTI // --- // DNMT3B /// microcosm // ENST0000...,NaN,NaN,NaN,hsa-miR-369-5p,0.003104,NaN
2360,20502360,MIMAT0001536_st,MIMAT0001536,hsa-miR-429,miRNA,Homo sapiens,chr1:1104435-1104456 (+),22,UAAUACUGUCUGGUAAAACCGU,---,hsa-mir-200b // chr1:1102484-1102578 (+) /// h...,MTI // --- // MYC /// microcosm // ENST0000000...,NaN,NaN,NaN,hsa-miR-429,0.007773,NaN
4557,20504557,MIMAT0003882_st,MIMAT0003882,hsa-miR-767-5p,miRNA,Homo sapiens,chrX:151561953-151561975 (-),23,UGCACCAUGGUUGUCUGAGCAUG,ENST00000370311 // sense // intron // 1 /// EN...,hsa-mir-105-1 // chrX:151560691-151560771 (-) ...,microcosm // ENST00000005374 // C7orf24 /// mi...,NaN,NaN,NaN,hsa-miR-767-5p,0.006993,NaN
6844,20506844,MIMAT0005903_st,MIMAT0005903,hsa-miR-1251-5p,miRNA,Homo sapiens,chr12:97885691-97885711 (+),21,ACUCUAGCUGCCAAAGGCGCU,ENST00000408552 // sense // exon // 1 /// ENST...,---,---,miRNA-4_0,"May 13, 2014",miRBase,hsa-miR-1251-5p,0.009721,NaN
7743,20507743,MIMAT0007348_st,MIMAT0007348,hsa-miR-1470,miRNA,Homo sapiens,chr19:15560359-15560379 (+),21,GCCCUCCGCCCGUGCACCCCG,ENST00000263381 // antisense // intron // 1 //...,---,---,miRNA-4_0,"May 13, 2014",miRBase,hsa-miR-1470,0.009135,NaN
15609,20515609,MIMAT0015057_st,MIMAT0015057,hsa-miR-3180-5p,miRNA,Homo sapiens,chr16:15005094-15005118 (+) /// chr16:16403749...,25,CUUCCAGACGCUCCGCCCCACGUCG,ENST00000548268 // antisense // exon // 1 /// ...,hsa-mir-3180-3 // chr16:18496035-18496128 (-) ...,---,miRNA-4_0,"May 13, 2014",miRBase,hsa-miR-3180-5p,0.002045,NaN
17745,20517745,MIMAT0016916_st,MIMAT0016916,hsa-miR-4286,miRNA,Homo sapiens,chr8:10524498-10524514 (+),17,ACCCCACUCCUGGUACC,ENST00000329335 // antisense // intron // 1 //...,---,MTI // --- // PDCD11,miRNA-4_0,"May 13, 2014",miRBase,hsa-miR-4286,0.000047,NaN
17755,20517755,MIMAT0027032_st,MIMAT0027032,hsa-miR-500b-3p,miRNA,Homo sapiens,chrX:49775330-49775349 (+),20,GCACCCAGGCAAGGAUUCUG,ENST00000376088 // sense // intron // 3 /// EN...,hsa-mir-188 // chrX:49768109-49768194 (+) /// ...,---,miRNA-4_0,"May 13, 2014",miRBase,hsa-miR-500b-3p,0.001586,NaN
18433,20518433,MIMAT0027036_st,MIMAT0027036,hsa-miR-3912-5p,miRNA,Homo sapiens,chr5:170813716-170813736 (-),21,AUGUCCAUAUUAUGGGUUAGU,---,---,---,miRNA-4_0,"May 13, 2014",miRBase,hsa-miR-3912-5p,0.002591,NaN


To determine if each gene is upregulated or downregulated, we compare the mean values against the control.

In [36]:
regulations = np.mean(samples, axis=-1) > np.mean(controls, axis=-1)
# True if upregulated, False if downregulated
signif_regulations = regulations[f_stats.pvalue < alpha]

labeled_sig_regs = np.stack([
    signifs['Transcript ID(Array Design)'].to_numpy(), signif_regulations], axis=-1)
labeled_sig_regs[mirna_mask]

array([['hsa-miR-215-3p', True],
       ['hsa-miR-369-5p', False],
       ['hsa-miR-429', True],
       ['hsa-miR-767-5p', False],
       ['hsa-miR-1251-5p', False],
       ['hsa-miR-1470', True],
       ['hsa-miR-3180-5p', False],
       ['hsa-miR-4286', False],
       ['hsa-miR-500b-3p', False],
       ['hsa-miR-3912-5p', True],
       ['hsa-miR-3929', True],
       ['hsa-miR-4540', True],
       ['hsa-miR-4633-3p', True],
       ['hsa-miR-4653-5p', True],
       ['hsa-miR-203b-5p', True],
       ['hsa-miR-4791', False],
       ['hsa-miR-5003-3p', False],
       ['hsa-miR-5093', False],
       ['hsa-miR-6877-5p', False],
       ['hsa-miR-7155-3p', False]], dtype=object)